In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.layers import Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split



drive.mount('/content/gdrive')

df_price = pd.read_csv(os.path.join('/content/gdrive/My Drive/Colab Notebooks', '발전량예측_경주.csv'), encoding='utf8')
df_forecast = pd.read_csv(os.path.join('/content/gdrive/My Drive/Colab Notebooks', '예보데이터_경주.csv'), encoding='cp949')




In [ ]:
def make_dataset(data, label, window_size=3):
    
    feature_list = []
    label_list = []
  
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

#설비이용률이 10%이상인 것들만 필터링. 경주 설비용량 1502.22   (발전량/설비용량) * 100 > 10


pd.to_datetime(df_price['일자'], format='%Y-%m-%d-%H')
pd.to_datetime(df_forecast['일자'], format='%Y-%m-%d-%H')
# 0      2020-01-07
# 1      2020-01-06
# 2      2020-01-03
# 3      2020-01-02
# 4      2019-12-30

df_price['일자'] = pd.to_datetime(df_price['일자'], format='%Y-%m-%d-%H')
df_forecast['일자'] = pd.to_datetime(df_forecast['일자'], format='%Y-%m-%d-%H')
#df_price['연도'] =df_price['일자'].dt.year
#df_price['월'] =df_price['일자'].dt.month
#df_price['일'] =df_price['일자'].dt.day
#df_price.describe()
#df_price = df_price.loc[((df_price['발전량']/1502.22) * 100) >= 10]
#df_forecast = df_forecast.loc[((df_forecast['발전량']/1502.22) * 100) >= 10]


df_generation = df_price[-50:]   #그래프를 좀 더 효휼적으로 보기 위해 범위를 한정했음.

plt.figure(figsize=(16, 9))
sns.lineplot(y=df_generation['발전량'], x=df_generation['일자'])
plt.xlabel('time')
plt.ylabel('power generation')




In [ ]:
df_price = df_price.sort_index(ascending=True).reset_index(drop=True)

In [ ]:
df_forecast = df_forecast.sort_index(ascending=True).reset_index(drop=True)

In [ ]:
df_generation2 = df_forecast[-50:]
plt.figure(figsize=(16, 9))
sns.lineplot(y=df_generation2['발전량'], x=df_generation2['일자'])
plt.xlabel('time')
plt.ylabel('power generation')

In [ ]:
df_forecast

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# df_price.sort_index(ascending=False).reset_index(drop=True)
# df_forecast.sort_index(ascending=False).reset_index(drop=True)

#scaler = MinMaxScaler()                                #전체 데이터는 0, 1사이의 값을 갖도록 해줍니다.
#scale_cols = ['발전량', '기온', '습도', '이슬점온도', '일사량', '전운량']   #일자	년	월	일	시	발전량	기온	습도	이슬점온도	일사량	전운량
#df_scaled = scaler.fit_transform(df_price[scale_cols])
#df_scaled = pd.DataFrame(df_scaled)
#df_scaled.columns = scale_cols
#df_scaled
df_price

In [ ]:
#train = df_scaled[:-TEST_SIZE]           #0~ max-200. training data.
#test = df_scaled[-TEST_SIZE:]            #max-200 부터 끝까지. 테스트용
train = df_price
test = df_forecast

feature_cols = ['월', '일', '시','기온', '습도', '이슬점온도', '일사량', '전운량']
label_cols = ['발전량']

train_feature = train[feature_cols]
train_label = train[label_cols]

test_feature = test[feature_cols]
test_label = test[label_cols]

#train_example = df_price[-2000:]
#train_example_label = train_example[label_cols]    #그래프를 좀 더 잘 볼 수 있도록 범위를 한정함

#ax = train_example_label.plot()
#test_label.plot(ax=ax)
#plt.legend(['train'])

In [ ]:

# train dataset
train_feature_array, train_label_array = make_dataset(train_feature, train_label)


# train, validation set 생성
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train_feature_array, train_label_array, test_size=0.2) #test_size는 테스트 데이터셋의 비율. 디폴트는 0.25


test_feature_array, test_label_array = make_dataset(test_feature, test_label)
test_feature.shape, test_label.shape

In [ ]:

model = Sequential()
model.add(LSTM(64, 
               input_shape=(train_feature_array.shape[1], train_feature_array.shape[2]),  #timestep과 feature
               activation='relu', 
               return_sequences=False)
          )
#model.add(Dropout(0.2)) #드롭아웃 추가해봣음.,..
model.add(Dense(1))   #Dense = 전결합층(fully-conntected layer)을 추가. 예측하고자 하는 target의 개수가 하나

model.compile(loss='mean_squared_error', optimizer='adam')   #모델 학습과정 설정.  optimizer : 훈련 과정 설정.  loss : 최적화 과정에서 최소화될 손실 함수(loss function) 설정
                                                             # optimizer 는 일반적으로 adam 사용

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=20)   #epoch 을 정하는데 많이 사용되는 Earlystopping 은 무조건 Epoch 을 많이 돌린 후, 특정 시점에서 멈추는 것이다. 
                                                              # 10회 Epoch동안 개선되지 않는다면 종료

filename = os.path.join('/content/gdrive/My Drive/Colab Notebooks', 'generation_checkpoint.h5')  

In [ ]:
checkpoint = ModelCheckpoint(filename,        #모델이 학습하면서 정의한 조건을 만족했을 때 Model의 weight 값을 중간 저장해 줌
                             monitor='val_loss',                    # val_loss 값이 개선되었을때 호출
                             verbose=1,                             # 로그를 출력합니다
                             save_best_only=True,                   # 가장 best 값만 저장합니다
                             mode='auto')                          

In [ ]:
# history = model.fit(x_train, y_train,      #fit 메소드를 통해서 훈련
#                     epochs=250,            #에포크. 한 에포크는 전체 입력 데이터를 한번 순회하는 것
#                     batch_size=32,
#                     validation_data=(x_valid, y_valid), 
#                    )                    
   
history = model.fit(x_train, y_train,      
                    epochs=250,           
                    batch_size=32,
                    validation_data=(x_valid, y_valid), 
                    callbacks=[early_stop, checkpoint]) #개선되지 않는 학습에 대한 조기 종료를 실행하고, ModelCheckpoint로 부터 가장 best model을 다시 로드하여 학습을 재게할 수 있음.


In [ ]:
from sklearn.metrics import mean_squared_error
#model.load_weights(filename)

# 예측
pred = model.predict(test_feature_array)

pred.shape
rmse = mean_squared_error(test_label_array , pred, squared=False)
rmse

In [ ]:
df_forecast = df_forecast.drop(index = [0,1,2,3,4], axis=0)
df_forecast = df_forecast.sort_index(ascending=True).reset_index(drop=True)
#예측오차율 = (abs(예측발전량 - 실제발전량) / 설비용량) * 100
df_forecast['예측발전량'] = pd.DataFrame(pred)
df_forecast['예측오차율'] = (abs(df_forecast['예측발전량'] - df_forecast['발전량']) / 1502.22  ) * 100
#df_price['예측오차율'] = df_forecast['예측발전량','발전량']


In [ ]:
pd.DataFrame(pred)

In [ ]:
df_forecast = df_forecast.dropna()


In [ ]:

mean_error = df_forecast['예측오차율'].mean()
mean_error


In [ ]:
df_forecast[:50]


In [ ]:


plt.figure(figsize=(16, 9))

forprint_test = test_label_array[-168:]
forprint_pred = pred[-168:]

#pd.DataFrame({'Real':test_label,'Predicted':pred}).plot()
plt.plot(forprint_test,label='actual')
plt.plot(forprint_pred, label='prediction')

plt.xticks(np.arange(0, 168, 24), labels=['2021-02-05', '2021-02-06', '2021-02-07', '2021-02-08', '2020-02-09', '2021-02-10', '2021-02-11', '2021-02-12'])
plt.legend()
plt.show()